In [1]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.6/595.6 kB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [29]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [2]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 2.7.0
google-cloud-aiplatform==1.70.0
google_cloud_pipeline_components version: 2.17.0


In [3]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components.v1.model import ModelUploadOp
from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,ModelDeployOp)
from google_cloud_pipeline_components.types import artifact_types

In [4]:

#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "de2024-435509"
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://temp_de2024_bz"   # e.g., gs://temp_de2024

In [18]:
@dsl.component(
    packages_to_install=["pandas", "scikit-learn==1.3.2"],
    base_image="python:3.10.7-slim"
)
def train_test_split(dataset: Input[Dataset], dataset_train: Output[Dataset], dataset_test: Output[Dataset]):
    '''train_test_split'''
    import pandas as pd
    import logging 
    import sys
    from sklearn.model_selection import train_test_split as tts
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO) 
    
    alldata = pd.read_csv(dataset.path, index_col=None)
    train, test = tts(alldata, test_size=0.3)
    train.to_csv(dataset_train.path + ".csv" , index=False, encoding='utf-8-sig')
    test.to_csv(dataset_test.path + ".csv" , index=False, encoding='utf-8-sig')

In [19]:
@dsl.component(
    packages_to_install=["pandas", "scikit-learn==1.3.2"],
    base_image="python:3.10.7-slim"
)
def train_rf(features: Input[Dataset], model: Output[Model]):
    '''Train a RandomForestClassifier using message text as features and label as the target'''
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.pipeline import Pipeline
    from sklearn.feature_extraction.text import TfidfVectorizer
    import pickle
    
    # Load the dataset, handling encoding for any special characters
    data = pd.read_csv(features.path + ".csv", encoding="utf-8")
    
    # Separate features and target
    X = data['message']    # Message column as the feature
    y = data['label']      # Label column as the target
    
    # Define the pipeline: text vectorization and classifier
    pipeline = Pipeline([
        ('vectorizer', TfidfVectorizer()),     # Convert text to TF-IDF features
        ('classifier', RandomForestClassifier())  # Train the RandomForestClassifier
    ])
    
    # Fit the pipeline to the data
    pipeline.fit(X, y)
    
    # Save metadata and the model
    model.metadata["framework"] = "RandomForest"
    file_name = model.path + f".pkl"
    with open(file_name, 'wb') as file:  
        pickle.dump(pipeline, file)


In [25]:
@dsl.component(
    packages_to_install=[
        "pandas", "scikit-learn==1.3.2", "numpy"
    ], base_image="python:3.10.7-slim"
)
def rf_model_evaluation(
    test_set: Input[Dataset],
    model_rf: Input[Model],
    thresholds_dict_str: str,
    metrics: Output[ClassificationMetrics],
    kpi: Output[Metrics]
) -> NamedTuple('outputs', approval=bool):
  
    import pandas as pd
    from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score
    import json
    import pickle
    from numpy import nan_to_num
    
    def threshold_check(val1, val2):
        return val1 >= val2

    # Load the test dataset
    data = pd.read_csv(test_set.path + ".csv")
    
    # Load the saved pipeline (with both vectorizer and classifier)
    m_filename = model_rf.path + ".pkl"
    model = pickle.load(open(m_filename, 'rb'))
    
    # Prepare test data
    X_test = data['message']  # Pass only 'message' column as features
    y_target = data['label']  # 'label' is the target column
    
    # Predict and calculate scores
    y_pred = model.predict(X_test)
    y_scores = model.predict_proba(X_test)[:, 1]
 
    # Calculate ROC curve values
    fpr, tpr, thresholds = roc_curve(
         y_true=y_target, y_score=y_scores, pos_label=True
    )
    thresholds = nan_to_num(thresholds)   
    metrics.log_roc_curve(fpr, tpr, thresholds)  
    
    # Log confusion matrix
    metrics.log_confusion_matrix(
        ['Negative', 'Positive'],
         confusion_matrix(y_target, y_pred).tolist(), 
    )
    
    # Calculate and log accuracy
    accuracy = accuracy_score(y_target, y_pred)
    thresholds_dict = json.loads(thresholds_dict_str)
    model_rf.metadata["accuracy"] = float(accuracy)
    kpi.log_metric("accuracy", float(accuracy))
    
    # Approval check based on the threshold
    outputs = NamedTuple('outputs', approval=bool)
    approval_value = threshold_check(float(accuracy), int(thresholds_dict['roc']))
    return outputs(approval_value)


In [26]:
@dsl.component(
    packages_to_install=["google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def upload_model_to_gcs(project_id: str, model_repo: str, model: Input[Model]):
    '''upload model to gsc'''
    from google.cloud import storage   
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)    
  
    # upload the model to GCS
    client = storage.Client(project=project_id)
    bucket = client.bucket(model_repo)
    blob = bucket.blob('model.pkl')
    source_file_name= model.path + '.pkl'
   
    blob.upload_from_filename(source_file_name)    
    
    print(f"File {source_file_name} uploaded to {model_repo}.")

In [27]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="sms-predictor-training-pipeline")
def pipeline(project_id: str, data_bucket: str, dataset_uri: str, model_repo: str, thresholds_dict_str:str, model_repo_uri:str):    
    
    dataset_op = kfp.dsl.importer(
        artifact_uri=dataset_uri,
        artifact_class=Dataset,
        reimport=False,
    )
     
    train_test_split_op = train_test_split(dataset=dataset_op.output)
        
    training_lr_job_run_op = train_rf(features=train_test_split_op.outputs["dataset_train"])
    
    model_evaluation_op = rf_model_evaluation(
        test_set=train_test_split_op.outputs["dataset_test"],
        model_rf=training_lr_job_run_op.outputs["model"],
        thresholds_dict_str=thresholds_dict_str, # I deploy the model anly if the model performance is above the threshold
    )
    
    with dsl.If(
        model_evaluation_op.outputs["approval"]== True,
        name="approve-model",
    ):
        upload_model_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_lr_job_run_op.outputs['model']
        )    
        
        import_unmanaged_model_task = dsl.importer(
            artifact_uri=model_repo_uri,
            artifact_class=artifact_types.UnmanagedContainerModel,
            metadata={
                "containerSpec": {
                    "imageUri": "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest",  # see https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers  
                },
            },
        ).after(upload_model_to_gc_op)      
       
        # using Google's custom components for for uloading and deploying the model.
       
        model_upload_op = ModelUploadOp(
            project=project_id,
            display_name="sms-prediction-model",
            unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
        ).after(import_unmanaged_model_task)       
               
        create_endpoint_op = EndpointCreateOp(
            project=project_id,
            display_name="sms-prediction-service",
        ).after(model_upload_op)      
        
        model_deploy_op = ModelDeployOp(
            model=model_upload_op.outputs["model"],
            endpoint=create_endpoint_op.outputs['endpoint'],
            deployed_model_display_name="sms-prediction-model",
            dedicated_resources_machine_type="n1-standard-4",
            dedicated_resources_min_replica_count=1,
            dedicated_resources_max_replica_count=1,
            traffic_split={"0": 100},
        ).after(create_endpoint_op)     

In [28]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='sms_prdictor_training_pipeline.yaml')

In [29]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(
    project=PROJECT_ID,
    location=REGION,
)

# Prepare the pipeline job
job = aip.PipelineJob(
    display_name="sms-predictor",
    enable_caching=False,
    template_path="sms_prdictor_training_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    location=REGION,
    parameter_values={
        'project_id': PROJECT_ID, # makesure to use your project id 
        'data_bucket': 'data_de2024_bz',  # makesure to use your data bucket name 
        'dataset_uri':'gs://data_de2024_bz/SMS_preporcessed.csv',
        'model_repo':'models_de2024_bz', # makesure to use your model bucket name 
        'thresholds_dict_str':'{"roc":0.8}',
        'model_repo_uri':'gs://models_de2024_bz' # makesure to use your model bucket name 
    }
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/471717444362/locations/us-central1/pipelineJobs/sms-predictor-training-pipeline-20241025090114
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/471717444362/locations/us-central1/pipelineJobs/sms-predictor-training-pipeline-20241025090114')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/sms-predictor-training-pipeline-20241025090114?project=471717444362
PipelineJob projects/471717444362/locations/us-central1/pipelineJobs/sms-predictor-training-pipeline-20241025090114 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/471717444362/locations/us-central1/pipelineJobs/sms-predictor-training-pipeline-20241025090114 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/471717444362/locations/us-central1/pipelineJobs/sms-predictor-training-pipeline-20241025090114 current state:
PipelineStat